# 15. 어텐션 메커니즘 (Attention Mechanism)

## 3) 양방향 LSTM과 어텐션 메커니즘(BiLSTM with Attention mechanism)

양방향 LSTM과 어텐션 메커니즘으로 IMDB 리뷰 감성 분류하기를 수행해봅시다.

### 1. IMDB 리뷰 데이터 전처리하기**

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

In [4]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


데이터 패딩

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### 2. 바다나우 어텐션(Bahdanau Attention)

바다나우 어텐션은 아래와 같은 어텐션 스코어 함수를 사용합니다.

$ score(query,\ key) = V^Ttanh(W_{1}key + W_{2}query) $

In [6]:
import tensorflow as tf

In [7]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, values, query): # 단, key와 value는 같음
        # query shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

### 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [8]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

입력층과 임베딩층을 설계합니다.

In [9]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, 
                               mask_zero=True)(sequence_input)

양방향 LSTM을 설계합니다.

In [10]:
# 첫번째 층입니다. 
# 두번째 층을 위에 쌓을 예정이므로 return_sequences를 True로 해주어야 합니다.
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

# 두번째 층을 설계합니다. 
# 상태를 리턴받아야 하므로 return_state를 True로 해주어야 합니다.
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

각 상태의 크기(shape)를 출력해보겠습니다.

In [11]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

In [12]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

어텐션 메커니즘에서는 은닉 상태를 사용합니다. 이를 입력으로 컨텍스트 벡터(context vector)를 얻습니다.

In [13]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

컨텍스트 벡터를 밀집층(dense layer)에 통과시키고, 이진 분류이므로 최종 출력층에 1개의 뉴런을 배치하고, 활성화 함수로 시그모이드 함수를 사용합니다.

In [14]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=3, batch_size=256, 
                    validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 [==============================] - 981s 10s/step - loss: 0.4622 - accuracy: 0.7744 - val_loss: 0.3111 - val_accuracy: 0.8672
Epoch 2/3
98/98 [==============================] - 946s 10s/step - loss: 0.2401 - accuracy: 0.9172 - val_loss: 0.2918 - val_accuracy: 0.8839
Epoch 3/3
98/98 [==============================] - 952s 10s/step - loss: 0.1831 - accuracy: 0.9392 - val_loss: 0.2958 - val_accuracy: 0.8800


In [17]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 619s 792ms/step - loss: 0.2958 - accuracy: 0.8800

 테스트 정확도: 0.8800
